In [ ]:
import numpy as np
import os
import cv2
import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
import albumentations as A
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
class ColorHistogramExtractor:   
    # Data Augmentation
    def augment_image(image):
        transform = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.3),
            A.Rotate(limit=30, p=0.5),
            A.RandomBrightnessContrast(p=0.3),
            A.GaussianBlur(p=0.3),
            A.CoarseDropout(max_holes=8, max_height=8, max_width=8, p=0.3),
        ])
        return transform(image=image)['image']
    def extract_color_histogram(image, bins=(x, y, z)):
        hist_lab = cv2.calcHist([cv2.cvtColor(image, cv2.COLOR_BGR2LAB)], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
        hist_hsv = cv2.calcHist([cv2.cvtColor(image, cv2.COLOR_BGR2HSV)], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
        hist_bgr = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
        
        cv2.normalize(hist_lab, hist_lab)
        cv2.normalize(hist_hsv, hist_hsv)
        cv2.normalize(hist_bgr, hist_bgr)
        
        return np.hstack([hist_lab.flatten(), hist_hsv.flatten(), hist_bgr.flatten()])
    
    def extract_contour_features(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if not contours:
            return np.zeros(3, dtype=np.float32), image  # Trả về ảnh gốc nếu không có contour
        
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]  # Giữ lại tối đa 5 contour lớn nhất
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, contours, -1, 255, thickness=cv2.FILLED)
        segmented_image = cv2.bitwise_and(image, image, mask=mask)
        
        areas = [cv2.contourArea(c) for c in contours]
        perimeters = [cv2.arcLength(c, True) for c in contours]
        
        return np.array([sum(areas), sum(perimeters), len(contours)], dtype=np.float32), segmented_image
    
    def extract_hu_moments(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        moments = cv2.moments(gray)
        hu_moments = cv2.HuMoments(moments).flatten()
        return -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)


NameError: name 'x' is not defined

In [ ]:
import numpy as np
import os
import cv2
import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
import albumentations as A
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# Data Augmentation
def augment_image(image):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.Rotate(limit=30, p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.GaussianBlur(p=0.3),
        A.CoarseDropout(max_holes=8, max_height=8, max_width=8, p=0.3),
    ])
    return transform(image=image)['image']

# Feature Extraction
def extract_color_histogram(image, bins=(8, 8, 8)):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    hist_lab = cv2.calcHist([lab], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256]).flatten()
    hist_hsv = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256]).flatten()
    
    return np.hstack([hist_lab, hist_hsv])

# Contour Feature Extraction with Canny Edge Detection
def extract_contour_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return np.zeros(3, dtype=np.float32)
    
    largest_contour = max(contours, key=cv2.contourArea)
    return np.array([cv2.contourArea(largest_contour), cv2.arcLength(largest_contour, True), len(contours)], dtype=np.float32)

# Hu Moments Feature Extraction
def extract_hu_moments(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    moments = cv2.moments(gray)
    hu_moments = cv2.HuMoments(moments).flatten()
    return -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)

image_paths = glob.glob("/kaggle/input/mushrooms-classification-common-genuss-images/Mushrooms/*/*.jpg")
labels = [os.path.basename(os.path.dirname(path)) for path in image_paths]

X, y = [], []
for path, label in tqdm(zip(image_paths, labels), total=len(image_paths), desc="Extracting Features"):
    image = cv2.imread(path)
    if image is None:
        continue
    
    color_hist = extract_color_histogram(image)
    contour_feat = extract_contour_features(image)
    hu_moments = extract_hu_moments(image)
    features = np.hstack([color_hist, contour_feat, hu_moments])
    
    X.append(features)
    y.append(label)
    
    for _ in range(5):  
        aug_img = augment_image(image)
        aug_color_hist = extract_color_histogram(aug_img)
        aug_contour_feat = extract_contour_features(aug_img)
        aug_hu_moments = extract_hu_moments(aug_img)
        aug_features = np.hstack([aug_color_hist, aug_contour_feat, aug_hu_moments])
        X.append(aug_features)
        y.append(label)

X = np.array(X, dtype=object)
y = np.array(y)

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Handle NaN
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Feature Selection with LGBM
lgb_feature_selector = lgb.LGBMClassifier()
lgb_feature_selector.fit(X_train, y_train)
feature_importances = lgb_feature_selector.feature_importances_
top_250_idx = np.argsort(feature_importances)[-250:]
X_train = X_train[:, top_250_idx]
X_test = X_test[:, top_250_idx]

# Train and evaluate models
rf = RandomForestClassifier(n_estimators=333, max_depth=40, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")

xgb_model = xgb.XGBClassifier(eval_metric='mlogloss', use_label_encoder=False)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
print(f"XGBoost Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")

lgb_model = lgb.LGBMClassifier()
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)
print(f"LightGBM Accuracy: {accuracy_score(y_test, y_pred_lgb):.4f}")

# Plot feature importance
sorted_idx = np.argsort(feature_importances)[-100:]
plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importances[sorted_idx], y=[f"Feature {i}" for i in sorted_idx])
plt.xlabel("Feature Importance")
plt.ylabel("Feature Index")
plt.title("Important Features - LGBM")
plt.show()